In [1]:
%pip install valentine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.3/682.3 KB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━

# CONTROLLO MATCHING

In [48]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased, Cupid
import pprint

# Load data using pandas
df1 = pd.read_json("schema_mediato_confronto.json")
# df2 = pd.read_json("datasets/output_globaldata.json")
df2 = pd.read_json("datasets/valueToday2.json")


# Instantiate matcher and run
# Coma requires java to be installed on your machine
# If java is not an option, all the other algorithms are in Python (e.g., Cupid)
# matcher = Coma(strategy="COMA_OPT")
matcher = DistributionBased(0.9, 0.9)
matches = valentine_match(df1, df2, matcher)
# print(type(matches))
# print(matches)
pp = pprint.PrettyPrinter(indent=4)

In [49]:
pp.pprint(matches)

{   (('table_1', 'country'), ('table_2', 'country')): 0.9626358793204958,
    (('table_1', 'employees'), ('table_2', 'world_rank')): 0.7102028743045415,
    (('table_1', 'name'), ('table_2', 'name')): 0.9264797282763465,
    (('table_1', 'sector'), ('table_2', 'country')): 0.865730815920655}


# CREAZIONE SCHEMA MEDIATO

In [5]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased
import pprint

In [6]:
# ---------------------------------------------------------
# CREATA PER INIZIALIZZARE UN DATASET FITTIZIO MA ORA NON
# SI UTILIZZA IL DATASET FITTIZIO DA NOI CREATO MANUALMENTE
# ---------------------------------------------------------
def creation_dataset_confronto(df1):
  # Load data using pandas
  df2 = pd.read_json("datasets/output_disfold.json")

  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])

  for i in df2.index:
      data = {}
      # print(i)
      for d,m in zip(attributi_dataset,attributi_mediato):
        # print(df2[d][i])
        data[m] = df2[d][i]
      df1=df1.append(data, ignore_index=True)

  return df1


In [28]:
def creation_df_log_matcher(df_logmachers, matches, i, type):
  record = {}
  record['tab1'] = i[0][0]
  record['attr1'] = i[0][1]
  record['tab2'] = i[1][0]
  record['attr2'] = i[1][1]
  record['prob'] = matches[i]
  if type == 'coma':
    record['matcher'] = 'Coma'
  else:
    record['matcher'] = 'DistributionBased'
  df_logmachers=df_logmachers.append(record, ignore_index=True)
  return df_logmachers

In [31]:
def creation_schema_alignment(df1, df2, df_confronto, df_logmachers):

  # Instantiate matcher and run
  # Coma requires java to be installed on your machine
  # If java is not an option, all the other algorithms are in Python (e.g., Cupid)
  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  distr_run = False

 
  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    df_logmachers = creation_df_log_matcher(df_logmachers, matches_coma, i, 'coma')
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])
    else:
      if not distr_run:
        matcher_distr = DistributionBased(0.9, 0.9)
        matches_distr = valentine_match(df_confronto, df2, matcher_distr)
        distr_run = True
      try:
        df_logmachers = creation_df_log_matcher(df_logmachers, matches_distr, i, 'distribution')
        if matches_distr[i] > 0.5:
          attributi_mediato.append(i[0][1])
          attributi_dataset.append(i[1][1])
      except:
        pass
  # print(attributi_dataset)
  # print(attributi_mediato)

  # inserimento dei dati nel dataframe
  for i in df2.index:
    data = {}
    # print(i)
    for d,m in zip(attributi_dataset,attributi_mediato):
      # print(df2[d][i])
      data[m] = df2[d][i]
    df1=df1.append(data, ignore_index=True)

  pp = pprint.PrettyPrinter(indent=4)

  # stampa dei matchers
  pp.pprint(matches_coma)
  if distr_run:
    pp.pprint(matches_distr)

  return df1, df_logmachers
  # print(df1)


In [55]:
from pathlib import Path
#formati json, csv, xls, xlsx, jsonl
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'share_price', 'website', 'stock', 'area_served', 'earnings'])
df_confronto = creation_dataset_confronto(df1)
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])

#numero di errori nell'esecuzione di file
num_error = 0

directory = 'datasets'
dataset_list_json = Path(directory).glob('*.json')
dataset_list_jsonl = Path(directory).glob('*.jsonl')
dataset_list_csv = Path(directory).glob('*.csv')
dataset_list_xls = Path(directory).glob('*.xls')
dataset_list_xlsx = Path(directory).glob('*.xlsx')
i = 0

# --------------------------------------------------------------------
# Modificare seleziona_tipo per scegliere che tipo di file eseguire
# --------------------------------------------------------------------
seleziona_tipo = 2

# CON SELEZIONA_TIPO=0 SI RUNNANO I TUTTI TIPI DI FILE
# CON SELEZIONA_TIPO=1 SI RUNNANO I JSON
if seleziona_tipo==0 or seleziona_tipo==1:
  for path in dataset_list_json:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=2 SI RUNNANO I JSONL
if seleziona_tipo==0 or seleziona_tipo==2:
  for path in dataset_list_jsonl:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path, lines=True)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=3 SI RUNNANO I CSV
if seleziona_tipo==0 or seleziona_tipo==3:
  for path in dataset_list_csv:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_csv(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=4 SI RUNNANO I XLS
if seleziona_tipo==0 or seleziona_tipo==4:
  for path in dataset_list_xls:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=5 SI RUNNANO I XLSX
if seleziona_tipo==0 or seleziona_tipo==5:
  for path in dataset_list_xlsx:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


print("Numero errori: " + str(num_error))
display(df1)
print('--------DATAFRAME LOG MATCHERS--------')
display(df_logmachers)

0 datasets/companiesMarketCap_dataset.jsonl
ERRORE datasets/companiesMarketCap_dataset.jsonl
1 datasets/hitHorizons_dataset.jsonl
{   (('table_1', 'address'), ('table_2', 'address')): 0.81608355,
    (('table_1', 'earnings'), ('table_2', 'sic_code')): 0.34194565,
    (('table_1', 'founded'), ('table_2', 'id')): 0.35452425,
    (('table_1', 'industry'), ('table_2', 'industry')): 0.8169457,
    (('table_1', 'name'), ('table_2', 'name')): 0.8130995}
{}
2 datasets/disfold_dataset.jsonl
{   (('table_1', 'ceo'), ('table_2', 'ceo')): 0.85876805,
    (('table_1', 'employees'), ('table_2', 'employees')): 0.86376804,
    (('table_1', 'founded'), ('table_2', 'founded')): 0.8629059,
    (('table_1', 'headquarters'), ('table_2', 'headquarters_country')): 0.66289085,
    (('table_1', 'market_cap'), ('table_2', 'market_cap')): 0.8653305,
    (('table_1', 'name'), ('table_2', 'name')): 0.8599219}
3 datasets/valueToday_dataset.jsonl
ERRORE datasets/valueToday_dataset.jsonl
Numero errori: 2


,name,industry,market_cap,employees,country,headquarters,address,sector,ceo,founded,share_price,website,stock,area_served,earnings
0,Enel Spa,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIALE REGINA MARGHERITA 137, ROMA, 00198, ROMA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Esselunga Spa,Retail Trade,NaN,NaN,NaN,NaN,"VIA GIAMBOLOGNA 1, PIOLTELLO, 20096, MILANO, I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Superit Srl,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIA VITTOR PISANI 20, MILANO, 20124, MILANO, I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Saras Spa,Manufacturing,NaN,NaN,NaN,NaN,"GALLERIA PASSARELLA 2, MILANO, 20122, MILANO, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Unicredit Spa,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"PIAZZA GAE AULENTI 3 TOWER A, MILANO, 20154, M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,Asahi Group Holdings Ltd,NaN,¥2.231 Trillion,"30,020",NaN,Tokyo,NaN,NaN,Mr. Atsushi Katsuki,NaN,NaN,NaN,NaN,NaN,NaN
1663,Haitong Securities Co. Ltd,NaN,CN¥115.98 Billion,"11,282",NaN,Shanghai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1664,"Catalent, Inc.",NaN,$18.24 Billion,"17,300",NaN,Somerset,NaN,NaN,Mr. John R. Chiminski,2007,NaN,NaN,NaN,NaN,NaN
1665,"Quanta Services, Inc.",NaN,$18.25 Billion,"43,700",NaN,Houston,NaN,NaN,Mr. Earl C. Austin Jr.,1997,NaN,NaN,NaN,NaN,NaN


--------DATAFRAME LOG MATCHERS--------


,tab1,attr1,tab2,attr2,prob,matcher
0,table_1,industry,table_2,industry,0.816946,Coma
1,table_1,address,table_2,address,0.816084,Coma
2,table_1,name,table_2,name,0.813099,Coma
3,table_1,founded,table_2,id,0.354524,Coma
4,table_1,earnings,table_2,sic_code,0.341946,Coma
5,table_1,market_cap,table_2,market_cap,0.865331,Coma
6,table_1,employees,table_2,employees,0.863768,Coma
7,table_1,founded,table_2,founded,0.862906,Coma
8,table_1,name,table_2,name,0.859922,Coma
9,table_1,ceo,table_2,ceo,0.858768,Coma


In [51]:
df1.to_csv('schema_mediato.csv')

In [52]:
df_logmachers.to_csv('log_matchers.csv')

# Il codice sottostante è utilizzato per generare il dataframe di un singolo dataset

In [37]:
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'share_price', 'website', 'stock', 'area_served', 'revenue'])
df_confronto = pd.read_json('schema_mediato_confronto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])


# df2 = pd.read_json('datasets/output_globaldata.json')
df2 = pd.read_json('datasets/valueToday2.json')
df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers)

df1
df_logmachers

{   (('table_1', 'ceo'), ('table_2', 'CEO')): 0.82983994,
    (('table_1', 'country'), ('table_2', 'country')): 0.8339778,
    (('table_1', 'market_cap'), ('table_2', 'marketCap')): 0.7074935,
    (('table_1', 'name'), ('table_2', 'name')): 0.8309938,
    (('table_1', 'stock'), ('table_2', 'world_rank')): 0.32348847}
{(('table_1', 'country'), ('table_2', 'country')): 0.9626358793204958}


,tab1,attr1,tab2,attr2,prob,matcher
0,table_1,country,table_2,country,0.833978,Coma
1,table_1,name,table_2,name,0.830994,Coma
2,table_1,ceo,table_2,CEO,0.829840,Coma
3,table_1,market_cap,table_2,marketCap,0.707493,Coma
4,table_1,stock,table_2,world_rank,0.323488,Coma
